In [1]:
from __future__ import print_function, division
import scipy

from keras.datasets import mnist
from keras_contrib.layers.normalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
from data_loader import DataLoader
import numpy as np
import os



## sigin
import signal
import sys

Using TensorFlow backend.


In [2]:
## source from keras-gan

class colorizeSketch():
    def __init__(self):
        # Input shape
        self.img_rows = 512
        self.img_cols = 512
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        # Configure data loader
        self.dataset_name = 'anime'
        self.data_loader = DataLoader(dataset_name=self.dataset_name,
                                      img_res=(self.img_rows, self.img_cols))


        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.gf = 16
        self.df = 16

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminators
        self.d_A = self.build_discriminator()
        self.d_B = self.build_discriminator()
        self.d_A.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])
        self.d_B.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        #-------------------------
        # Construct Computational
        #   Graph of Generators
        #-------------------------

        # Build the generators
        self.g_AB = self.build_generator()
        self.g_BA = self.build_generator()

        # Input images from both domains
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # Translate images to the other domain
        fake_B = self.g_AB(img_A)
        fake_A = self.g_BA(img_B)
        # Translate images back to original domain
        reconstr_A = self.g_BA(fake_B)
        reconstr_B = self.g_AB(fake_A)

        # For the combined model we will only train the generators
        self.d_A.trainable = False
        self.d_B.trainable = False

        # Discriminators determines validity of translated images
        valid_A = self.d_A(fake_A)
        valid_B = self.d_B(fake_B)

        # Objectives
        # + Adversarial: Fool domain discriminators
        # + Translation: Minimize MAE between e.g. fake B and true B
        # + Cycle-consistency: Minimize MAE between reconstructed images and original
        self.combined = Model(inputs=[img_A, img_B],
                              outputs=[ valid_A, valid_B,
                                        fake_B, fake_A,
                                        reconstr_A, reconstr_B ])
        self.combined.compile(loss=['mse', 'mse',
                                    'mae', 'mae',
                                    'mae', 'mae'],
                              optimizer=optimizer)
        

    def build_generator(self):
        """U-Net Generator"""

        def conv2d(layer_input, filters, f_size=4, normalize=True):
            """Layers used during downsampling"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if normalize:
                d = InstanceNormalization()(d)
            return d

        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            u = InstanceNormalization()(u)
            u = Concatenate()([u, skip_input])
            return u

        # Image input
        d0 = Input(shape=self.img_shape)

        # Downsampling
        d1 = conv2d(d0, self.gf, normalize=False)
        d2 = conv2d(d1, self.gf*2)
        d3 = conv2d(d2, self.gf*4)
        d4 = conv2d(d3, self.gf*8)
        d5 = conv2d(d4, self.gf*8)
        d6 = conv2d(d5, self.gf*8)
        d7 = conv2d(d6, self.gf*8)

        # Upsampling
        u1 = deconv2d(d7, d6, self.gf*8)
        u2 = deconv2d(u1, d5, self.gf*8)
        u3 = deconv2d(u2, d4, self.gf*8)
        u4 = deconv2d(u3, d3, self.gf*4)
        u5 = deconv2d(u4, d2, self.gf*2)
        u6 = deconv2d(u5, d1, self.gf)

        u7 = UpSampling2D(size=2)(u6)
        output_img = Conv2D(self.channels, kernel_size=4, strides=1,
                            padding='same', activation='tanh')(u7)

        return Model(d0, output_img)

    def build_discriminator(self):

        def d_layer(layer_input, filters, f_size=4, normalization=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if normalization:
                d = InstanceNormalization()(d)
            return d

        img = Input(shape=self.img_shape)

        d1 = d_layer(img, self.df, normalization=False)
        d2 = d_layer(d1, self.df*2)
        d3 = d_layer(d2, self.df*4)
        d4 = d_layer(d3, self.df*8)

        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50,load=False):

        start_time = datetime.datetime.now()


        if load == True:
            1+1
        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)
        
        
        Gloss=100
        for epoch in range(epochs):

            for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(batch_size)):

                # ----------------------
                #  Train Discriminators
                # ----------------------

                # Translate images to opposite domain
                fake_B = self.g_AB.predict(imgs_A)
                fake_A = self.g_BA.predict(imgs_B)

                # Train the discriminators (original images = real / translated = Fake)
                dA_loss_real = self.d_A.train_on_batch(imgs_A, valid)
                dA_loss_fake = self.d_A.train_on_batch(fake_A, fake)
                dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

                dB_loss_real = self.d_B.train_on_batch(imgs_B, valid)
                dB_loss_fake = self.d_B.train_on_batch(fake_B, fake)
                dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

                # Total disciminator loss
                d_loss = 0.5 * np.add(dA_loss, dB_loss)

                # ------------------
                #  Train Generators
                # ------------------

                # Train the generators
                g_loss = self.combined.train_on_batch([imgs_A, imgs_B], [valid, valid, \
                                                                         imgs_B, imgs_A, \
                                                                         imgs_A, imgs_B])

                elapsed_time = datetime.datetime.now() - start_time
                # Plot the progress
                print ("[%d] [%d/%d] time: %s, [d_loss: %f, g_loss: %f]" % (epoch, batch_i,
                                                                        self.data_loader.n_batches,
                                                                        elapsed_time,
                                                                        d_loss[0], g_loss[0]))

                
                # If at save interval => save generated image samples
                try:
                    if batch_i % 4 == 0 and epoch % 10 == 0  :
                        self.sample_images(epoch, batch_i,g_loss[0])
                    if g_loss[0] < Gloss and epoch > 1 :
                        self.g_BA.save('./models/gen_model3.h5')
                        self.g_BA.save_weights('./models/gen_weights3.h5')
                        self.g_AB.save('./models/dis_model3.h5')
                        self.g_AB.save_weights('./models/dis_weights3.h5')
                        self.combined.save('./models/model3.h5')
                        self.combined.save_weights('./models/weights3.h5')
                        Gloss = g_loss[0] 
                except :
                    print()
                    

    def sample_images(self, epoch, batch_i,g_loss):
        os.makedirs('images/%s' % self.dataset_name, exist_ok=True)
        r, c = 2, 3

        imgs_A, imgs_B = self.data_loader.load_data(batch_size=1, is_testing=True)

        # Translate images to the other domain
        fake_B = self.g_AB.predict(imgs_A)
        fake_A = self.g_BA.predict(imgs_B)
        # Translate back to original domain
        reconstr_A = self.g_BA.predict(fake_B)
        reconstr_B = self.g_AB.predict(fake_A)

        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Original', 'Translated:'+str(round(g_loss,4))+"%", 'Reconstructed']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[j])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%s/%d_%d.png" % (self.dataset_name, epoch, batch_i))
        plt.close()

def signal_handler(self,sig, frame):
        print('Graceful Interupt')


In [ ]:

 ## dis learning false

gan = colorizeSketch()
    
gan.train(epochs=200, batch_size=2, sample_interval=26,load=False)


C:\Users\me\Anaconda3\envs\tf-gpu\lib\site-packages\keras\engine\training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[0] [0/3684] time: 0:01:05.157806, [d_loss: 3.079710, g_loss: 6.994631]
[0] [1/3684] time: 0:01:07.425065, [d_loss: 1.083668, g_loss: 6.334970]
[0] [2/3684] time: 0:01:08.670762, [d_loss: 2.349969, g_loss: 7.688252]
[0] [3/3684] time: 0:01:09.899940, [d_loss: 0.916632, g_loss: 5.601794]
[0] [4/3684] time: 0:01:11.162784, [d_loss: 3.889068, g_loss: 6.143694]
[0] [5/3684] time: 0:01:13.078742, [d_loss: 1.214282, g_loss: 5.635256]
[0] [6/3684] time: 0:01:14.331034, [d_loss: 0.826424, g_loss: 4.133498]
[0] [7/3684] time: 0:01:15.590620, [d_loss: 0.392701, g_loss: 4.251064]
[0] [8/3684] time: 0:01:16.854195, [d_loss: 0.428544, g_loss: 3.346460]
[0] [9/3684] time: 0:01:18.749746, [d_loss: 0.316694, g_loss: 5.564346]
[0] [10/3684] time: 0:01:19.980655, [d_loss: 0.475527, g_loss: 3.177059]
[0] [11/3684] time: 0:01:21.232295, [d_loss: 0.630163, g_loss: 2.526632]
[0] [12/3684] time: 0:01:22.479731, [d_loss: 0.434556, g_loss: 2.820538]
[0] [13/3684] time: 0:01:24.404260, [d_loss: 0.549049, g_loss

[0] [113/3684] time: 0:03:45.469064, [d_loss: 0.323322, g_loss: 1.665618]
[0] [114/3684] time: 0:03:46.696678, [d_loss: 0.351841, g_loss: 1.398090]
[0] [115/3684] time: 0:03:47.922556, [d_loss: 0.350643, g_loss: 1.435428]
[0] [116/3684] time: 0:03:49.164433, [d_loss: 0.356342, g_loss: 1.266059]
[0] [117/3684] time: 0:03:51.076870, [d_loss: 0.325767, g_loss: 1.476035]
[0] [118/3684] time: 0:03:52.320915, [d_loss: 0.384001, g_loss: 1.249900]
[0] [119/3684] time: 0:03:53.588560, [d_loss: 0.354520, g_loss: 1.272433]
[0] [120/3684] time: 0:03:54.800987, [d_loss: 0.340274, g_loss: 1.373186]
[0] [121/3684] time: 0:03:56.695502, [d_loss: 0.333298, g_loss: 1.274239]
[0] [122/3684] time: 0:03:57.923486, [d_loss: 0.363382, g_loss: 1.287202]
[0] [123/3684] time: 0:03:59.176231, [d_loss: 0.324199, g_loss: 1.455593]
[0] [124/3684] time: 0:04:00.410458, [d_loss: 0.319320, g_loss: 1.298786]
[0] [125/3684] time: 0:04:02.306447, [d_loss: 0.376448, g_loss: 1.132157]
[0] [126/3684] time: 0:04:03.552466, [

[0] [224/3684] time: 0:06:21.894004, [d_loss: 0.321037, g_loss: 1.240621]
[0] [225/3684] time: 0:06:23.812830, [d_loss: 0.314626, g_loss: 1.318928]
[0] [226/3684] time: 0:06:25.041759, [d_loss: 0.327119, g_loss: 0.994343]
[0] [227/3684] time: 0:06:26.280601, [d_loss: 0.310468, g_loss: 1.298152]
[0] [228/3684] time: 0:06:27.532502, [d_loss: 0.314048, g_loss: 1.086267]
[0] [229/3684] time: 0:06:29.438565, [d_loss: 0.278692, g_loss: 1.313661]
[0] [230/3684] time: 0:06:30.687637, [d_loss: 0.289382, g_loss: 1.425012]
[0] [231/3684] time: 0:06:31.918243, [d_loss: 0.320773, g_loss: 1.148436]
[0] [232/3684] time: 0:06:33.166474, [d_loss: 0.310048, g_loss: 1.393804]
[0] [233/3684] time: 0:06:35.074261, [d_loss: 0.307781, g_loss: 1.351791]
[0] [234/3684] time: 0:06:36.323330, [d_loss: 0.293207, g_loss: 1.308362]
[0] [235/3684] time: 0:06:37.572416, [d_loss: 0.290327, g_loss: 1.258548]
[0] [236/3684] time: 0:06:38.810021, [d_loss: 0.316765, g_loss: 1.252929]
[0] [237/3684] time: 0:06:40.682011, [

[0] [335/3684] time: 0:08:58.204265, [d_loss: 0.346855, g_loss: 1.545746]
[0] [336/3684] time: 0:08:59.421789, [d_loss: 0.386451, g_loss: 1.228819]
[0] [337/3684] time: 0:09:01.326149, [d_loss: 0.327445, g_loss: 1.389387]
[0] [338/3684] time: 0:09:02.574781, [d_loss: 0.319211, g_loss: 1.770771]
[0] [339/3684] time: 0:09:03.808470, [d_loss: 0.303929, g_loss: 1.285629]
[0] [340/3684] time: 0:09:05.013478, [d_loss: 0.285325, g_loss: 1.296901]
[0] [341/3684] time: 0:09:06.923878, [d_loss: 0.307023, g_loss: 1.086870]
[0] [342/3684] time: 0:09:08.175657, [d_loss: 0.275846, g_loss: 1.396008]
[0] [343/3684] time: 0:09:09.415207, [d_loss: 0.299990, g_loss: 1.089810]
[0] [344/3684] time: 0:09:10.664100, [d_loss: 0.299662, g_loss: 1.123134]
[0] [345/3684] time: 0:09:12.553021, [d_loss: 0.285036, g_loss: 1.134887]
[0] [346/3684] time: 0:09:13.793816, [d_loss: 0.303931, g_loss: 1.318072]
[0] [347/3684] time: 0:09:15.012247, [d_loss: 0.287106, g_loss: 1.155101]
[0] [348/3684] time: 0:09:16.249145, [

...

[1] [3379/3684] time: 2:39:57.885454, [d_loss: 0.238166, g_loss: 1.122228]
[1] [3380/3684] time: 2:39:59.151525, [d_loss: 0.254485, g_loss: 1.141010]
[1] [3381/3684] time: 2:40:00.429598, [d_loss: 0.257362, g_loss: 0.818059]
[1] [3382/3684] time: 2:40:01.730682, [d_loss: 0.212949, g_loss: 1.275064]
[1] [3383/3684] time: 2:40:02.974754, [d_loss: 0.232405, g_loss: 1.178289]
[1] [3384/3684] time: 2:40:04.220815, [d_loss: 0.256866, g_loss: 1.200840]
[1] [3385/3684] time: 2:40:05.480900, [d_loss: 0.239946, g_loss: 1.250624]
[1] [3386/3684] time: 2:40:06.757967, [d_loss: 0.259077, g_loss: 1.196488]
[1] [3387/3684] time: 2:40:08.019031, [d_loss: 0.228880, g_loss: 1.191148]
[1] [3388/3684] time: 2:40:09.316104, [d_loss: 0.291044, g_loss: 1.107396]
[1] [3389/3684] time: 2:40:10.623181, [d_loss: 0.242403, g_loss: 1.051610]
[1] [3390/3684] time: 2:40:11.891256, [d_loss: 0.297858, g_loss: 1.232696]
[1] [3391/3684] time: 2:40:13.156326, [d_loss: 0.274706, g_loss: 0.953918]
[1] [3392/3684] time: 2:4

[1] [3489/3684] time: 2:42:17.483533, [d_loss: 0.285333, g_loss: 1.153873]
[1] [3490/3684] time: 2:42:18.754602, [d_loss: 0.264477, g_loss: 1.055418]
[1] [3491/3684] time: 2:42:20.002675, [d_loss: 0.301619, g_loss: 1.140757]
[1] [3492/3684] time: 2:42:21.279760, [d_loss: 0.285656, g_loss: 1.137791]
[1] [3493/3684] time: 2:42:22.552822, [d_loss: 0.269179, g_loss: 1.247140]
[1] [3494/3684] time: 2:42:23.807893, [d_loss: 0.261374, g_loss: 1.053929]
[1] [3495/3684] time: 2:42:25.053964, [d_loss: 0.255160, g_loss: 1.071078]
[1] [3496/3684] time: 2:42:26.333052, [d_loss: 0.275551, g_loss: 1.038501]
[1] [3497/3684] time: 2:42:27.613112, [d_loss: 0.237690, g_loss: 1.235774]
[1] [3498/3684] time: 2:42:28.872197, [d_loss: 0.257217, g_loss: 1.061794]
[1] [3499/3684] time: 2:42:30.114254, [d_loss: 0.238076, g_loss: 1.106016]
[1] [3500/3684] time: 2:42:31.384339, [d_loss: 0.252222, g_loss: 1.198659]
[1] [3501/3684] time: 2:42:32.641414, [d_loss: 0.242036, g_loss: 1.128636]
[1] [3502/3684] time: 2:4

[1] [3599/3684] time: 2:44:36.398868, [d_loss: 0.256530, g_loss: 1.065730]
[1] [3600/3684] time: 2:44:37.659763, [d_loss: 0.242025, g_loss: 1.113816]
[1] [3601/3684] time: 2:44:38.916838, [d_loss: 0.238387, g_loss: 1.264453]
[1] [3602/3684] time: 2:44:40.186920, [d_loss: 0.288921, g_loss: 1.158099]
[1] [3603/3684] time: 2:44:41.517013, [d_loss: 0.284068, g_loss: 1.302384]
[1] [3604/3684] time: 2:44:42.896075, [d_loss: 0.246449, g_loss: 1.135712]
[1] [3605/3684] time: 2:44:44.178139, [d_loss: 0.248522, g_loss: 1.136731]
[1] [3606/3684] time: 2:44:45.446658, [d_loss: 0.236789, g_loss: 1.194807]
[1] [3607/3684] time: 2:44:46.705740, [d_loss: 0.243574, g_loss: 1.178397]
[1] [3608/3684] time: 2:44:47.945798, [d_loss: 0.241261, g_loss: 1.540897]
[1] [3609/3684] time: 2:44:49.218874, [d_loss: 0.270294, g_loss: 1.162061]
[1] [3610/3684] time: 2:44:50.455961, [d_loss: 0.251809, g_loss: 1.162324]
[1] [3611/3684] time: 2:44:51.723015, [d_loss: 0.256603, g_loss: 1.224041]
[1] [3612/3684] time: 2:4

[2] [27/3684] time: 2:47:17.780727, [d_loss: 0.240096, g_loss: 1.103940]
[2] [28/3684] time: 2:47:19.051803, [d_loss: 0.217503, g_loss: 1.198673]
[2] [29/3684] time: 2:47:20.301865, [d_loss: 0.263907, g_loss: 0.911363]
[2] [30/3684] time: 2:47:22.488002, [d_loss: 0.235045, g_loss: 1.193436]
[2] [31/3684] time: 2:47:23.750062, [d_loss: 0.186476, g_loss: 1.329318]
[2] [32/3684] time: 2:47:25.002134, [d_loss: 0.221467, g_loss: 1.266559]
[2] [33/3684] time: 2:47:26.259204, [d_loss: 0.231142, g_loss: 1.417798]
[2] [34/3684] time: 2:47:27.513304, [d_loss: 0.237626, g_loss: 1.601176]
[2] [35/3684] time: 2:47:28.779350, [d_loss: 0.279184, g_loss: 1.309110]
[2] [36/3684] time: 2:47:30.022213, [d_loss: 0.312172, g_loss: 1.115998]
[2] [37/3684] time: 2:47:31.267285, [d_loss: 0.280499, g_loss: 1.345588]
[2] [38/3684] time: 2:47:32.541355, [d_loss: 0.286722, g_loss: 0.902286]
[2] [39/3684] time: 2:47:34.739862, [d_loss: 0.261797, g_loss: 1.214811]
[2] [40/3684] time: 2:47:35.986933, [d_loss: 0.2774

[2] [139/3684] time: 2:49:40.885028, [d_loss: 0.246366, g_loss: 1.288095]
[2] [140/3684] time: 2:49:42.126100, [d_loss: 0.253122, g_loss: 1.224168]
[2] [141/3684] time: 2:49:43.375186, [d_loss: 0.238340, g_loss: 1.057621]
[2] [142/3684] time: 2:49:44.647252, [d_loss: 0.234377, g_loss: 1.119423]
[2] [143/3684] time: 2:49:45.894330, [d_loss: 0.216398, g_loss: 1.507604]
[2] [144/3684] time: 2:49:47.156410, [d_loss: 0.218181, g_loss: 1.304870]
[2] [145/3684] time: 2:49:48.424476, [d_loss: 0.252623, g_loss: 1.287875]
[2] [146/3684] time: 2:49:49.693549, [d_loss: 0.254474, g_loss: 1.327382]
[2] [147/3684] time: 2:49:50.932611, [d_loss: 0.250125, g_loss: 1.129487]
[2] [148/3684] time: 2:49:52.191681, [d_loss: 0.259587, g_loss: 1.225730]
[2] [149/3684] time: 2:49:53.457756, [d_loss: 0.301338, g_loss: 1.244942]
[2] [150/3684] time: 2:49:54.726828, [d_loss: 0.262030, g_loss: 1.388399]
[2] [151/3684] time: 2:49:55.976914, [d_loss: 0.243509, g_loss: 1.461447]
[2] [152/3684] time: 2:49:57.238974, [